In [4]:
import pandas as pd
import duckdb
import os
from pathlib import Path
import numpy as np
import glob

## Merging data sources

Let us consider we have different datasources that we want to join by a given column.
For example the two following csv files

In [23]:
duckdb.read_csv('./data/flights3.csv')

┌────────────┬────────────────┬─────────────────┐
│ FlightDate │ OriginCityName │  DestCityName   │
│    date    │    varchar     │     varchar     │
├────────────┼────────────────┼─────────────────┤
│ 1988-01-01 │ New York, NY   │ Los Angeles, CA │
│ 1988-01-02 │ New York, NY   │ Los Angeles, CA │
└────────────┴────────────────┴─────────────────┘

In [28]:
duckdb.read_csv('./data/flights4.csv')

┌────────────┬───────────────┬────────────────┬─────────────────┐
│ FlightDate │ UniqueCarrier │ OriginCityName │  DestCityName   │
│    date    │    varchar    │    varchar     │     varchar     │
├────────────┼───────────────┼────────────────┼─────────────────┤
│ 1988-01-03 │ AA            │ New York, NY   │ Los Angeles, CA │
└────────────┴───────────────┴────────────────┴─────────────────┘

We can use the following query to join the two csv files

In [62]:
query_str = "SELECT * FROM read_csv(['./data/flights3.csv', './data/flights4.csv'], union_by_name = true, AUTO_DETECT=TRUE);"
df = duckdb.query(query_str)
df

┌────────────┬────────────────┬─────────────────┬───────────────┐
│ FlightDate │ OriginCityName │  DestCityName   │ UniqueCarrier │
│    date    │    varchar     │     varchar     │    varchar    │
├────────────┼────────────────┼─────────────────┼───────────────┤
│ 1988-01-01 │ New York, NY   │ Los Angeles, CA │ NULL          │
│ 1988-01-02 │ New York, NY   │ Los Angeles, CA │ NULL          │
│ 1988-01-03 │ New York, NY   │ Los Angeles, CA │ AA            │
└────────────┴────────────────┴─────────────────┴───────────────┘

We can store the relation table to disk

In [61]:
df.to_csv('./data/merge_flights3_flights4.csv', header=True)

And we can read it later (in pandas for example)

In [60]:
pd.read_csv('./data/merge_flights3_flights4.csv')

,FlightDate,OriginCityName,DestCityName,UniqueCarrier
0,1988-01-01,"New York, NY","Los Angeles, CA",NaN
1,1988-01-02,"New York, NY","Los Angeles, CA",NaN
2,1988-01-03,"New York, NY","Los Angeles, CA",AA


#### Merging parquet files, selecting only some columns

Let us consider we want to do the same as before but merging columns

In [68]:
duckdb.read_csv('./data/flights3.csv').to_parquet('./data/flights3.parquet')

In [72]:
duckdb.read_csv('./data/flights4.csv').to_parquet('./data/flights4.parquet')

We can do the same we did of joining the different parquet files

In [76]:
query_str = "SELECT * FROM read_parquet(['./data/flights3.parquet', './data/flights4.parquet'], union_by_name = true);"
df = duckdb.query(query_str)

In [77]:
df

┌────────────┬────────────────┬─────────────────┬───────────────┐
│ FlightDate │ OriginCityName │  DestCityName   │ UniqueCarrier │
│    date    │    varchar     │     varchar     │    varchar    │
├────────────┼────────────────┼─────────────────┼───────────────┤
│ 1988-01-01 │ New York, NY   │ Los Angeles, CA │ NULL          │
│ 1988-01-02 │ New York, NY   │ Los Angeles, CA │ NULL          │
│ 1988-01-03 │ New York, NY   │ Los Angeles, CA │ AA            │
└────────────┴────────────────┴─────────────────┴───────────────┘

In [83]:
duckdb.read_csv('./data/merge_flights3_flights4.csv').to_parquet('./data/merge_flights3_flights4.parquet')

In [85]:
ls data

flights3.csv                     flights4.parquet
flights3.parquet                 merge_flights3_flights4.csv
flights4.csv                     merge_flights3_flights4.parquet


We can read more than 2 files

In [96]:
query_str = "SELECT FlightDate, OriginCityName',  FROM read_parquet(['./data/flights3.parquet', './data/flights4.parquet', './data/merge_flights3_flights4.parquet'], union_by_name = true);"
df = duckdb.query(query_str)

In [97]:
df

┌────────────┬────────────────┐
│ FlightDate │ OriginCityName │
│    date    │    varchar     │
├────────────┼────────────────┤
│ 1988-01-01 │ New York, NY   │
│ 1988-01-02 │ New York, NY   │
│ 1988-01-03 │ New York, NY   │
│ 1988-01-01 │ New York, NY   │
│ 1988-01-02 │ New York, NY   │
│ 1988-01-03 │ New York, NY   │
└────────────┴────────────────┘

01_intro_ducdb.ipynb     data/                    requirements.txt
02_merging_data.ipynb    diary_asin_tagger.ipynb  test.csv
